R# Preprocesamiento de los comentarios para la primera fase de fine-tuning y entrenamiento del modelo
En esta primera fase, una vez extraídos los bodies de todos los issues/PRs de los repositorios listados junto con 2 comentarios de 250 del total de issues de cada repositorio, se prepararán todos los datos para ser utilizados en el entrenamiento y ajuste fino del modelo que se utilizará.

Se seguirán prácticamente los mismos pasos vistos en *c_preparing_data_for_statistics_and_ML* pero con varias diferencias claves que existen entre los modelos BERT que se utilizarán ahora y los modelos de clasificación presentados con anterioridad (notebooks de GVTIA).

Para Transformers funciona mejor un preprocesado mínimo y dejar la segmentación al propio tokenizador del modelo, a continuación se muestra que procedimientos similares a los anteriores se mantendrán y cuáles se evitarán:

## Se mantendrá:
- Normalización de espacios/saltos de línea
- Eliminación de caracteres de control raros o poco usuales
- Se conservará el uso de mayúsculas y minúsculas, signos, números, URLs, nombres propios de vulnerabilidades o bugs (CVE-XXXX-YYYY), rutas (/etc/...), código entre backticks (`return salida`), nombres de APIs.
- Se definirá una longitud máxima de tokens por comentario o el uso de *sliding window* si el texto es muy largo.

## Se omitirá:
- Pasar todo el texto a minúsculas, los modelos RoBERTa/DistilRoBERTa que se utilizarán utilizan mayúsculas y minúsculas.
- Eliminar la puntuación y stopwords.
- Stemming / lematización.
- Normalizaciones agresivas de URLs/código -> se pierde señal técnica.

Una vez explicado esto, se comenzará con el preprocesado de todos los comentarios extraídos de GitHub, comenzando como se ha visto ya en diversas ocasiones, con cargar el documento (.csv) en un dataframe de pandas para su uso y manipulación.

En este caso, se cuenta con 2 documentos:
- **gh_bodys_lastyear.csv**. Archivo que contiene los bodies (comentario principal) de todos los Issues/PRs en el último año de los repositorios listados para la extracción de comentarios.
- **gh_comments_lastyear.csv**. Archivo que contiene los 2 primeros comentarios de cada Issue/PR de 250 Issues/PRs por repositorio (500 comentarios por repo), en gran parte de los casos serán las respuestas aportadas por usuarios al body del documento anterior.

En este caso, como se cuenta con 2 documentos lo que se hará es crear 2 dataframes, uno con cada documento, para a continuación unirlos con la función `concat()` de pandas y ordenarlos según el id del Issue/PR para la clara visualización y mantener una estructura coherente entre cuerpo principal y comentarios asociados.

In [1]:
import sys
# Cargamos el archivo settings_bert.py que contiene todos los imports necesarios para la correcta ejecución del notebook y no tener que importarlos en cada celda de código
%run -i ../notebooks_settings/settings_bert.py

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
# Ruta de los archivos
path_gh_comments = "../data/gh_comments/training/gh_comments_2023_now.csv"
path_gh_comments = Path(path_gh_comments)

EXPECTED_COLS = [
    'repo','is_pr','issue_number','comment_type','comment_id','comment_created_at','comment_author',
    'text','comment_url','context_id','container_title','container_state','container_url',
    'container_created_at','container_updated_at','container_labels'
]

def read_with_header_fix(p: Path) -> pd.DataFrame:
    # Se lee 1ª fila para inspeccionar columnas
    probe = pd.read_csv(p, nrows=1)
    if list(probe.columns) == EXPECTED_COLS:
        return pd.read_csv(p)
    # Si no coincide, reinterpretamos: no hay cabecera -> header=None + names=EXPECTED_COLS
    return pd.read_csv(p, header=None, names=EXPECTED_COLS)

df_gh = read_with_header_fix(path_gh_comments)

# Tipos y ordenación
df_gh['comment_created_at'] = pd.to_datetime(df_gh['comment_created_at'], errors='coerce', utc=True)
df_gh.loc[df_gh['comment_created_at'].isna(), 'comment_created_at'] = pd.to_datetime(df_gh['container_created_at'], errors='coerce', utc=True)

order_map = {'issue_body':0, 'pr_body':0} # Bodies primero -> coherencia
df_gh['order'] = df_gh['comment_type'].map(order_map).fillna(1).astype(int)

df_gh = df_gh.sort_values(by=['repo','issue_number','order','comment_created_at','comment_id'], kind='mergesort').drop(columns=['order'])

# Normalizar booleano -> OPCIONAL
df_gh['is_pr'] = df_gh['is_pr'].astype(str).str.lower().map({'true':True, 'false':False})

len(df_gh)

100634

Ahora sí están todos los comentarios bien ordenados.

Ahora sí se procederá al procesamiento del dataset para dejarlo preparado para el modelo BERT que se utilizará, RoBERTa o DistilRoBERTa. Este proceso se va a definir en una serie de scripts .py, cada uno con el objetivo de realizar una tarea para su reutilización en otros puntos del proyecto (cuando se haga el de reddit, u otros comentarios de github) de forma que estos sean agnósticos al sistema del que se extraen los comentarios que serán utilizados por el modelo.

Del mismo modo, tras el procesamiento de los datos, el resultado del procesado será almacenado en archivos `.parquet` por su ligereza y agilidad a la hora de ser manipulados y consumidos por modelos BERT. Las principales ventajas de este formato son:
- Más pequeño: compresión por columna, pesa de 2 a 5 veces menos que un `.csv`
- Más rápido: lee solo las columnas que se necesitan ("column pruning") y aplica vectorización.
- Conserva tipos: fechas, booleanos, enteros "nullable", etc. (`.csv`los pierde)
- Esquema: guarda el _schema_ dentro del archivo -> menos sorpresas al cargarlo

Por estas características el formato es el preferido para pipelines de datos/ML. En este caso la estructura que se utilizará será:
- `merged.parquet` = será el dataset completo tras ingesta y normalización ligera.
- `split_train.parquet`, `split_dev.parquet`, `split_test.parquet` = particiones del dataset ya divididas, listas para su tokenización.

#### Ventajas frente a CSV
- Evita problemas de comas y saltos de línea
- Mantiene las fechas (`created_at`), booleanos (`is_pr`) y enteros sin perder el tipo.
- Carga solo las columnas necesarias -> menor uso de RAM y tiempo de ejecución.

## Ejecución del preprocesado de datos
Se han definido varios scripts, cada uno con una función en el preprocesado de datos para modelos BERT, a  los cuáles se les llamará desde este notebook con los argumentos correspondientes para realizar este proceso.

Las funciones reutilizables para el pipeline están definidas en `prep_utils.py`. Normaliza texto "ligero", mapea columnas heterogéneas (GH/Reddit) al esquema core (id, text, label, source, created_at, context_id), lee CSV/SQLite y guarda Parquet.
El objetivo principal de estas funciones es su utilización cuando se desea agnosticismo de fuente (Github/Reddit) y un preprocesado mínimo ideal para Transformers.

### Filtro de ruido (bots/plantilllas)

In [3]:
# prep_utils.py (función reutilizable)
BOT_REGEX = r"(bot$)|(\[bot\])|(github-actions)|(^k8s-ci-robot$)|(dependabot)|(^renovate)|(^mergify)"
TPL_PATTERNS = [
    "This issue is currently awaiting triage",
    "Instructions for interacting with me using PR comments are available here",
]

def is_noise(author: str, text: str) -> bool:
    a = (author or "").lower()
    t = (text or "").strip().lower()
    if re.search(BOT_REGEX, a): return True
    if len(t.split()) < 10: return True
    for p in TPL_PATTERNS:
        if p.lower() in t: return True
    return False

before = len(df_gh)
df_gh = df_gh[~df_gh.apply(lambda r: is_noise(str(r.get("comment_author","")), str(r.get("text",""))), axis=1)].reset_index(drop=True)
after = len(df_gh)

print(f"Filtrado ruido: {before-after} filas eliminadas / {after} restantes")

Filtrado ruido: 41439 filas eliminadas / 59195 restantes


### Construcción del esquema core (context, label, source, id, created_at)B

## Qué hace la celda

1. **`build_context(row)`**

   * Toma el **título** del issue/PR (`container_title`) y las **labels** (`container_labels`, separadas por `;`) y construye un campo de **contexto legible** para el modelo, con este formato:

     ```
     "<título> [label1][label2][label3]"
     ```
   * ¿Por qué? Ese “context” suele ayudar a los modelos de lenguaje a desambiguar el texto del comentario: el título resume el problema y las labels (p. ej. `sig/security`, `kind/bug`) añaden señal semántica muy útil.

2. **`BUG_HINTS` y `SEC_HINTS`**

   * Son conjuntos de **pistas** a nivel de label que indican, con heurística muy sencilla, si algo es “bug” o “security”. No es perfecto, pero vale para informes rápidos, estratificación de splits o para auditoría de cobertura.

3. **`weak_label(row)`**

   * Genera una **etiqueta débil** (`security`, `bug` u `other`) combinando:

     * La intersección de las labels del issue/PR con `SEC_HINTS`/`BUG_HINTS`.
     * Palabras clave en el **título** o el **texto** del comentario (p. ej. `cve-`, `vulnerability`, `rce`, `xss`, etc.).
   * Útil para:

     * Hacer **quick reports** (distribución por clases).
     * **Filtrar**/priorizar subconjuntos.
     * **Estratificar** splits train/dev/test manteniendo diversidad.
   * Si no vas a usar clasificación supervisada por clase, puedes seguir dejándolo para informes y diagnóstico, o quitarlo.

4. **Asignaciones al DataFrame**

   * `df_gh["context"]` → el contexto construido arriba.
   * `df_gh["label"]` → la etiqueta débil.
   * `df_gh["source"] = "<GITHUB>"` → columna con el **origen**.
   * `df_gh["id"] = df_gh["comment_id"]` → un **ID estable** para deduplicar/trazar.
   * `df_gh["created_at"]` → fecha normalizada como `datetime` UTC (útil para splits temporales y análisis).

5. **`cols_core` y `df_core`**

   * Proyecta a un **esquema core** consistente: `id, text, context, label, source, created_at, …`
   * Esto facilita reusar el mismo pipeline con otras fuentes en el futuro (o con nuevos extractores).

---

## ¿Por qué `source = "<GITHUB>"` si sólo usarás GitHub?

* **Estabilidad de esquema**: dejar la columna `source` evita re-escribir scripts cuando añadas otra fuente (o distintas variantes de GitHub).
* **Trazabilidad/reproducibilidad**: queda claro en los artefactos de datos de dónde provienen (útil en informes, auditoría y versionado).
* **Prefijo de dominio opcional**: si en el futuro mezclas fuentes, puedes activar en tu tokenizador un **prefijo de dominio** (`--use-domain-prefix`) para enseñar al modelo a distinguir estilos de lenguaje. Con una sola fuente, **simplemente no lo activas** y el campo no molesta.

> Si quieres, puedes quitarlo sin romper nada: elimina la línea que asigna `source`, y borra `"source"` de `cols_core`. Pero mi recomendación es **dejarlo**; no te penaliza y te ahorra cambios cuando amplíes el dataset.

---

## Resumen rápido de utilidad de cada campo

* **context**: aporta señal de alto nivel (título + labels) al modelo.
* **label** (débil): para análisis/estratificación; no es obligatorio para el fine-tuning si no haces clasificación por clase.
* **source**: trazabilidad y futura multi-fuente (inocuo si no lo usas).
* **id**: dedupe + trazabilidad.
* **created_at**: splits temporales y análisis por época.

Si quieres, te paso una variante “minimal” sin `label` y sin `source`; pero tal cual está te da más control sin añadir complejidad real.


In [4]:
# --- saneamiento previo para evitar NaN ---
for col in ["container_title", "container_labels", "text", "comment_author"]:
    if col not in df_gh.columns:
        df_gh[col] = ""
    df_gh[col] = df_gh[col].fillna("").astype(str)

def build_context(row) -> str:
    title = str(row.get("container_title") or "").strip()
    labels_raw = row.get("container_labels")
    try:
        labels = str(labels_raw).strip()
    except Exception:
        labels = ""
    labels_list = [s for s in labels.split(";") if s]
    labels_fmt = f"[{']['.join(labels_list)}]" if labels_list else ""
    return " ".join([title, labels_fmt]).strip()

BUG_HINTS = {"kind/bug", "sig/bug", "triage/accepted", "area/bug"}
SEC_HINTS = {"sig/security", "area/security", "kind/security"}

def weak_label(row) -> str:
    labs_str = str(row.get("container_labels") or "")
    labs = set(s.strip().lower() for s in labs_str.split(";") if s)
    title = str(row.get("container_title") or "").lower()
    text  = str(row.get("text") or "").lower()

    if (labs & SEC_HINTS) or any(k in title or k in text for k in ["cve-", "vulnerability", "exploit", "rce", "xss", "csrf", "ssrf"]):
        return "security"
    if (labs & BUG_HINTS) or ("bug" in title):
        return "bug"
    return "other"

df_gh["context"] = df_gh.apply(build_context, axis=1)
df_gh["label"]   = df_gh.apply(weak_label, axis=1)
df_gh["source"]  = "<GITHUB>"
df_gh["id"]      = df_gh["comment_id"]
df_gh["created_at"] = pd.to_datetime(df_gh["comment_created_at"], errors="coerce", utc=True)

cols_core = [
    "id","text","context","label","source","created_at",
    "repo","issue_number","is_pr","comment_type","context_id",
    "comment_author","comment_url","container_url"
]
df_core = df_gh[cols_core].copy()
df_core.head(3).T


,0,1,2
id,github_issuecomment_IC_kwDOAiVL486CLpLA,github_issuecomment_IC_kwDOAiVL486ZLicv,github_issuecomment_IC_kwDOAiVL485wtrJq
text,My code has one package.json in root and the other in release/app/package.json\nI tried changing the version in package.json but it does not change the version number.,From the discussion at develar/app-builder/issues/96 I gather this issue might not originate in electron-builder itself but rather in its dependency develar/app-builder. Can anyone confirm?,"Hello everyone. I encountered this issue myself a few days ago and I've figured out how to fix it.\nBasically, make sure any native files are not packed (not in the .asar file and instead in the a..."
context,extraResources get copied to the wrong folder on Linux/Windows [help wanted][invalid][question][readme improvement][backlog],RPM Failure [feature][linux],ELECTRON_ASAR.js:158 Uncaught Error: The specified module could not be found. \\?\C:\Users\HzPC\AppData\Local\Temp\937.tmp.node
label,security,other,other
source,<GITHUB>,<GITHUB>,<GITHUB>
created_at,2024-06-22 16:29:53+00:00,2025-01-03 23:43:01+00:00,2024-01-14 18:07:56+00:00
repo,electron-userland/electron-builder,electron-userland/electron-builder,electron-userland/electron-builder
issue_number,379,502,580
is_pr,False,False,False
comment_type,issue_comment,issue_comment,issue_comment


In [5]:
print(len(df_core))

# Guarda SQLite (opcional)
db_gh = "../data/gh_comments/training/gh_dataset_2023-2025.db"
Path(db_gh).parent.mkdir(parents=True, exist_ok=True)
con = sqlite3.connect(db_gh)
df_core.to_sql('gh_dataset', con, if_exists='replace', index=False)
con.close()

# Guarda Parquet (entrada directa para el pipeline)
merged_parquet = "../src/artifacts/prep/merged.parquet"
Path(merged_parquet).parent.mkdir(parents=True, exist_ok=True)
df_core.to_parquet(merged_parquet, index=False)
print("Guardado:", merged_parquet)


59195
Guardado: ../src/artifacts/prep/merged.parquet


### `ingest_merge.py`
Combina una o varias entradas en un único DataFrame, deduplica el contenido por `id`, aplica normalización ligera y guarda `merged.parquet` + meta. Se le da como input el archivo `.db` o los `.csv` deseados, produciendo en la salida un DF en formato `.parquet`.

In [6]:
base = "../src/data_prep"

# 1. Ingesta + merge (de CSVs o db en SQLite)
# Como ya tengo el dataset directamente almacenado en un .db
subprocess.run([sys.executable, f"{base}/ingest_merge.py",
                "--sqlite-db", "../data/gh_comments/training/gh_dataset_2023-2025.db",
                "--table", "gh_dataset",
                "--apply-noise-filter",
                "--build-core-schema",
                "--drop-empty-text",
                "--source-default", "<GITHUB>",
                "--out-parquet", "../src/artifacts/prep/merged.parquet",
                "--out-meta", "../src/artifacts/prep/merged_meta.json"
                ], check=True)

[drop-empty-text] 0 filas sin texto eliminadas
OK -> ../src/artifacts/prep/merged.parquet


CompletedProcess(args=['/home/diego/BERTolto/.venv/bin/python', '../src/data_prep/ingest_merge.py', '--sqlite-db', '../data/gh_comments/training/gh_dataset_2023-2025.db', '--table', 'gh_dataset', '--apply-noise-filter', '--build-core-schema', '--drop-empty-text', '--source-default', '<GITHUB>', '--out-parquet', '../src/artifacts/prep/merged.parquet', '--out-meta', '../src/artifacts/prep/merged_meta.json'], returncode=0)

### `quick_report.py`
Muestra por pantalla un resumen del Parquet aportado en el input: nº de filas, distribución de `label`, distribución de `source`, rango de `created_at`.
Es útil para la verificación visual de que la ingesta es correcta antes del split/tokenización e ideal para detectar desbalanceos fuertes o rangos temporales inesperados antes de la fase de entrenamiento.

In [7]:
# Informe rápido: añade labels/source + rango temporal + top repos
subprocess.run([sys.executable, f"{base}/quick_report.py",
                "--in-parquet", "../src/artifacts/prep/merged.parquet",
                "--topn", "12"
                ], check=True)

rows: 59195
labels: {'other': 31477, 'security': 19719, 'bug': 7999}
fuentes: {'<GITHUB>': 59195}
rango de fechas: 2023-01-01T03:34:12+00:00 -> 2025-10-24T18:44:34+00:00
top 12 repos:
  - grafana/grafana: 11743
  - kubernetes/kubernetes: 10703
  - vercel/next.js: 9726
  - tensorflow/tensorflow: 7563
  - nodejs/node: 4398
  - pytorch/pytorch: 4358
  - openssl/openssl: 3670
  - envoyproxy/envoy: 3648
  - prometheus/prometheus: 1426
  - kubernetes/ingress-nginx: 1101
  - electron-userland/electron-builder: 859


CompletedProcess(args=['/home/diego/BERTolto/.venv/bin/python', '../src/data_prep/quick_report.py', '--in-parquet', '../src/artifacts/prep/merged.parquet', '--topn', '12'], returncode=0)

### `split_thread_temporal.py`
Divide el DF `merged.parquet` en train/dev/test respetando hilos (`context_id`) y orden temporal para evitar fuga de información entre splits manteniendo el orden cronológico dentro de cada uno.
Los `ratios` son las proporciones con las que se dividirá el dataset:
- train: 70%
- dev: 15%
- test: 15%

In [8]:
# Split temporal + thread-aware
subprocess.run([sys.executable, f"{base}/split_thread_temporal.py",
                "--in-parquet", "../src/artifacts/prep/merged.parquet",
                "--out-train", "../src/artifacts/prep/split_train.parquet",
                "--out-dev", "../src/artifacts/prep/split_dev.parquet",
                "--out-test", "../src/artifacts/prep/split_test.parquet",
                "--ratios", "0.7", "0.15", "0.15",
                "--out-meta", "../src/artifacts/prep/split_meta.json"
                ], check=True)

OK -> ../src/artifacts/prep/split_train.parquet ../src/artifacts/prep/split_dev.parquet ../src/artifacts/prep/split_test.parquet


CompletedProcess(args=['/home/diego/BERTolto/.venv/bin/python', '../src/data_prep/split_thread_temporal.py', '--in-parquet', '../src/artifacts/prep/merged.parquet', '--out-train', '../src/artifacts/prep/split_train.parquet', '--out-dev', '../src/artifacts/prep/split_dev.parquet', '--out-test', '../src/artifacts/prep/split_test.parquet', '--ratios', '0.7', '0.15', '0.15', '--out-meta', '../src/artifacts/prep/split_meta.json'], returncode=0)

### `tokenize_hf.py`
Carga los Parquet de train/dev/test, antepone un prefijo de dominio (`<GITHUB>`, `<REDDIT>`, en este caso no porque todavía estoy solo con los comentarios de GH), tokeniza los textos con el tokenizador de DistilRoBERTa/RoBERTa y guarda:
- `dataset/` (formato `save_to_disk` de HF Datasets)
- `tokenizer/` (vocabulario + config)
- `preprocess_meta.json` (tamaños, `max_len`, pesos de cada clase)

#### Parámetros
- `--base-model`: modelo que se utilizará, en este caso distilroberta
- `--max-len 384`: longitud de secuencia; bajar acelera, subir captura más contexto.
- `--use-domain-prefix`: etiqueta de la fuente (GITHUB/REDDIT) -> No se utiliza al usar solo github
- `--sliding-window --slide-stride 128`: para textos muy largos (menos truncado)

Para decidir el `--max-len` y la posibilidad de utilizar `--sliding-window`, se puede calcular la distribución de longitudes en tokens y elegir el valor del argumento para cubrir el *p90-p95* y obtener un mejor resultado.

In [9]:
tok = AutoTokenizer.from_pretrained("distilroberta-base", use_fast=True)
df = pd.read_parquet("../src/artifacts/prep/merged.parquet") # o split_train.parquet
lens = df["text"].astype(str).map(lambda s: len(tok(s, truncation=False)["input_ids"]))

print(pd.Series(lens).describe(percentiles=[.5, .9, .95, .99]))

Token indices sequence length is longer than the specified maximum sequence length for this model (1527 > 512). Running this sequence through the model will result in indexing errors


count    59195.00
mean       732.62
std       3020.39
min         12.00
50%        124.00
90%       1022.00
95%       2188.00
99%      20218.90
max     126155.00
Name: text, dtype: float64


### Argumentos del tokenizador
- `--max-len 384`. Como el modelo solo puede leer hasta N tokens al mismo tiempo, se limita a N tokens (384) en la tokenización de cada texto. Si este tiene un nº superior se trunca o se divide en trozos (sliding-window). Elijo 384 tokens máx. por texto por su equilibrio en velocidad y uso de VRAM (no debería de tener problema con mi PC).
- `--sliding-window`. Si un texto no cabe en `--max-len`, en lugar de descartar el sobrante se divide en ventanas solapadas, dando como resultado varias filas por cada texto largo.
- `--slide-stride 128`. Es el número de tokens que se solapan entre ventanas para no cortar frases/palabras a la mitad cuando se divide un texto entre ventanas. En este caso si el texto se divide en 2 ventanas la distribución sería:
    - Ventana 1: tokens 1-384
    - Ventana 2: tokens (384-128) 256-639
- `--max-windows-per-doc 6`. Limita a un máximo de 6 ventanas por texto, el resto se recorta. Es mejor que descartar el texto completo, y si supera el máx. de ventanas es muy probable que sea debido a mensajes de logs o diffs y no aporte nada al modelo, solo ruido.
- `--filter-max-input-tokens`. Como se ve en la celda anterior (porcentajes) hay textos enormes y con este argumento se descartan sus colas limitando a N tokens antes de dividir en ventanas.
- `--num-proc 12`. Usa 12 hilos de ejecución para map paralelo (Mi PC tiene 8C/16T) -> Implementar en el tokenizador
- `--batch-chunk-size 512`. Indica el tamaño de trozos (grandes) en cada llamada al tokenizador. -> Implementar en el tokenizador

#### Recomendaciones
- Si ves que sube mucho el nº de ejemplos → baja --max-windows-per-doc a 4.
- Si va justo de RAM → --num-proc 8 y/o --batch-chunk-size 256.

In [11]:
env = dict(os.environ)

# Fuerza UTF-8 en el hijo:
env["PYTHONIOENCODING"] = "utf-8"
env["PYTHONUTF8"] = "1"

# Paralelización
env["TOKENIZERS_PARALLELISM"] = "false" #evita sub-proceso
env["RAYON_NUM_THREADS"] = "2" #2 hilos por proceso

# tokenización HF (DistilRoBERTa/RoBERTa)
cmd = [
    sys.executable, f"{base}/tokenize_hf.py",
    "--train-parquet", "../src/artifacts/prep/split_train.parquet",
    "--dev-parquet",   "../src/artifacts/prep/split_dev.parquet",
    "--test-parquet",  "../src/artifacts/prep/split_test.parquet",
    "--out-dir", "../src/artifacts/hf_distilroberta",
    "--base-model", "distilroberta-base",
    "--max-len", "384",
    "--sliding-window",
    "--slide-stride", "128",
    "--max-window-per-doc", "8",      # tu script acepta ambas variantes
    "--filter-max-input-tokens", "8192",
    "--num-proc", "8",
    "--batch-chunk-size", "512",
]

res = subprocess.run(cmd, env=env, capture_output=True, text=True)
print("---- STDOUT ----\n", res.stdout)
print("---- STDERR ----\n", res.stderr)
if res.returncode != 0:
    raise RuntimeError(f"tokenize_hf.py falló con exit code {res.returncode}")


---- STDOUT ----
 [filtro] train -> 725 descartados por > 8192 tokens
[filtro] validation -> 151 descartados por > 8192 tokens
[filtro] test -> 161 descartados por > 8192 tokens
OK -> ../src/artifacts/hf_distilroberta/dataset ../src/artifacts/hf_distilroberta/tokenizer

---- STDERR ----
 None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Token indices sequence length is longer than the specified maximum sequence length for this model (1142 > 384). Running this sequence through the model will result in indexing errors

Map (num_proc=8): 100%|██████████| 40719/40719 [00:05<00:00, 7029.47 examples/s]

Map (num_proc=8): 100%|██████████| 8736/8736 [00:02<00:00, 3927.90 examples/s]

Map (num_proc=8): 100%|██████████| 8703/8703 [00:02<00:00, 4082.08 examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 66831/66831 [00:00<00:00, 256638.01 examples/s]

Saving the dataset (1/1 sha

Cálculo del factor de expansión (filas tokenizadas / filas originales) -> Si se va demasiado, bajar --max-window a 4

In [13]:
# Carga tamaños "antes" (parquets de split)
split_train_path = "../src/artifacts/prep/split_train.parquet"
split_dev_path   = "../src/artifacts/prep/split_dev.parquet"
split_test_path  = "../src/artifacts/prep/split_test.parquet"

n_train_raw = len(pd.read_parquet(split_train_path))
n_dev_raw   = len(pd.read_parquet(split_dev_path))
n_test_raw  = len(pd.read_parquet(split_test_path))

# Carga dataset tokenizado "después"
from datasets import load_from_disk
tok_ds = load_from_disk("../src/artifacts/hf_distilroberta/dataset")

n_train_tok = len(tok_ds["train"])
n_dev_tok   = len(tok_ds["validation"])
n_test_tok  = len(tok_ds["test"])

exp_train = n_train_tok / n_train_raw
exp_dev   = n_dev_tok   / n_dev_raw
exp_test  = n_test_tok  / n_test_raw

print(f"Train: {n_train_raw} -> {n_train_tok} (x{exp_train:.2f})")
print(f"Dev:   {n_dev_raw} -> {n_dev_tok} (x{exp_dev:.2f})")
print(f"Test:  {n_test_raw} -> {n_test_tok} (x{exp_test:.2f})")

Train: 41444 -> 66831 (x1.61)
Dev:   8887 -> 16111 (x1.81)
Test:  8864 -> 15077 (x1.70)


**Sanity check** -> Comprobar que no hay basura
Decodificación de un ejemplo tokenizado

In [14]:
# Carga tokenizer guardado (el del artefacto, no el de huggingface por nombre)
tok = AutoTokenizer.from_pretrained("../src/artifacts/hf_distilroberta/tokenizer", use_fast=True)

# Ejemplo del split de train ya tokenizado
ex = tok_ds["train"][0]
decoded = tok.decode(ex["input_ids"], skip_special_tokens=True)
print(decoded[:500])

# Opcional: ver metadatos (mappings de labels si los guardaste)
meta_path = Path("../src/artifacts/hf_distilroberta/preprocess_meta.json")
if meta_path.exists():
    meta = json.loads(meta_path.read_text(encoding="utf-8"))
    print("\nMeta breve:", {k: meta[k] for k in ["base_model","max_len","sliding_window","slide_stride","counts"]})
    if "label_to_id" in meta and meta["label_to_id"]:
        print("label_to_id:", meta["label_to_id"])

Verify canary release

 I verified that the issue exists in the latest Next.js canary release

Provide environment information
Operating System:
 Platform: linux
 Arch: x64
 Version: #62-Ubuntu SMP Tue Nov 22 19:54:14 UTC 2022
Binaries:
 Node: 16.19.0
 npm: 8.19.3
 Yarn: N/A
 pnpm: 7.18.2
Relevant packages:
 next: 13.1.1
 eslint-config-next: 13.1.1
 react: 18.2.0
 react-dom: 18.2.0

Which area(s) of Next.js are affected? (leave empty if unsure)
App directory (appDir: true)
Link to the code that 

Meta breve: {'base_model': 'distilroberta-base', 'max_len': 384, 'sliding_window': True, 'slide_stride': 128, 'counts': {'train': 66831, 'validation': 16111, 'test': 15077}}
label_to_id: {'bug': 0, 'other': 1, 'security': 2}


**Velocidad** -> Si el tiempo de ejecución es muy elevado usar --max-len 256 o subir --slide-stride a 192 para menos ventanas